# UR5e Robot Connection (Using RTDE)

This notebook demonstrates how to connect to a UR5e robot arm using the RTDE (Real-Time Data Exchange) protocol.

## Why RTDE instead of urx?

**urx library:**
- Sends complete URScript programs for every command
- Slower communication (full scripts sent each time)
- Simpler API, good for basic applications
- Lower update frequency (~10-20 Hz)

**RTDE library (ur_rtde):**
- Official UR protocol for real-time communication
- **500 Hz update rate** for e-series robots (vs ~10-20 Hz for urx)
- Sends only data packets, not full scripts
- Better for real-time control, force control, and high-performance applications
- Recommended for UR5e (e-series) robots

**Bottom line:** RTDE is much faster and more efficient, especially for the UR5e.

## Install Required Libraries

First, install the ur_rtde library (official RTDE interface for Universal Robots):

```bash
pip install ur-rtde
```

In [1]:
import rtde_control
import rtde_receive
import time
import math

## Connect to UR5e Robot

Replace `ROBOT_IP` with your UR5e's actual IP address (e.g., '192.168.1.100')

**Note:** RTDE uses two separate interfaces:
- `RTDEReceiveInterface`: For reading data from the robot (500 Hz)
- `RTDEControlInterface`: For sending commands to the robot

In [2]:
# Set your robot's IP address
ROBOT_IP = "10.1.10.69"  # Change this to your robot's IP

try:
    # Connect to the robot
    # rtde_receive: reads robot state at 500 Hz (e-series)
    rtde_r = rtde_receive.RTDEReceiveInterface(ROBOT_IP)
    
    # rtde_control: sends control commands to robot
    rtde_c = rtde_control.RTDEControlInterface(ROBOT_IP)
    
    print(f"Successfully connected to UR5e at {ROBOT_IP}")
    
    # Get current joint positions (in radians)
    joints = rtde_r.getActualQ()
    print(f"\nCurrent joint positions (radians): {joints}")
    
    # Convert to degrees for easier reading
    joints_deg = [math.degrees(j) for j in joints]
    print(f"Current joint positions (degrees): {joints_deg}")
    
    # Get current TCP (Tool Center Point) position [x, y, z, rx, ry, rz]
    tcp_pos = rtde_r.getActualTCPPose()
    print(f"\nCurrent TCP position [x,y,z,rx,ry,rz]: {tcp_pos}")
    
    # Get additional robot state information
    print(f"\nRobot mode: {rtde_r.getRobotMode()}")
    print(f"Safety status: {rtde_r.getSafetyStatusBits()}")
    
    # Get current TCP speed
    tcp_speed = rtde_r.getActualTCPSpeed()
    print(f"Current TCP speed: {tcp_speed}")
    
except Exception as e:
    print(f"Error connecting to robot: {e}")

Successfully connected to UR5e at 10.1.10.69

Current joint positions (radians): [1.454850673675537, -0.7980327171138306, -0.09259447455406189, -0.6524940890124817, 4.639284610748291, -3.06081206003298]
Current joint positions (degrees): [83.35680342337285, -45.72390660398003, -5.305272598179242, -37.38515745764867, 265.81142815587003, -175.3716129226329]

Current TCP position [x,y,z,rx,ry,rz]: [0.04959030858362103, -0.6510723353812558, 0.669089883659722, -3.095756213364316, 0.31367354875572356, 0.11966895408890237]

Robot mode: 7
Safety status: 1
Current TCP speed: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


RTDEReceiveInterface boost system Exception: (asio.misc:2) End of file [asio.misc:2 at /Users/massimo/anaconda3/include/boost/asio/detail/reactive_socket_recv_op.hpp:134:37 in function 'do_complete']


## Example: Read Robot Data in Real-Time

One advantage of RTDE is the ability to read data at 500 Hz

In [ ]:
# Read robot data for 5 seconds
print("Reading robot data for 5 seconds...\n")

start_time = time.time()
sample_count = 0

while time.time() - start_time < 5.0:
    # Get actual joint positions
    joints = rtde_r.getActualQ()
    
    # Get TCP force (useful for force control)
    tcp_force = rtde_r.getActualTCPForce()
    
    sample_count += 1
    
    # Print every 100 samples to avoid flooding output
    if sample_count % 100 == 0:
        print(f"Sample {sample_count}: Joint 0 = {joints[0]:.4f} rad, TCP Force Z = {tcp_force[2]:.2f} N")
    
    time.sleep(0.002)  # 500 Hz = 2ms interval

print(f"\nTotal samples collected: {sample_count}")
print(f"Average sampling rate: {sample_count / 5.0:.1f} Hz")

In [46]:
home_joints = [1.4412965774536133, -1.2482660573771973, 0.6950395742999476, -0.9592366975596924, 4.640990257263184, -3.063166920338766]
#home_joints = current_joints
print(f"\nCurrent joint positions (radians): {home_joints}")
rtde_c.moveJ(home_joints, speed=0.5, acceleration=0.5)


Current joint positions (radians): [1.4412965774536133, -1.2482660573771973, 0.6950395742999476, -0.9592366975596924, 4.640990257263184, -3.063166920338766]


True

## Example: Move Robot (Safely)

Before running movement commands, ensure the robot workspace is clear!

**RTDE Movement Commands:**
- `moveJ()`: Joint space movement
- `moveL()`: Linear movement in Cartesian space
- `moveJ_IK()`: Joint movement using inverse kinematics from TCP pose
- `servoJ()`: Real-time joint servo (for smooth trajectories)
- `servoL()`: Real-time Cartesian servo

In [56]:
# Example: Small joint movement
# WARNING: Ensure workspace is clear before running!

# Get current position
current_joints = rtde_r.getActualQ()
print(f"Starting position: {current_joints}")

# Create a small movement (move joint 0 by 0.1 radians)
new_joints = list(current_joints)
new_joints[0] -= 0.0

# Move to new position with velocity and acceleration limits
# speed: joint speed in rad/s
# acceleration: joint acceleration in rad/s^2
rtde_c.moveJ(new_joints, speed=0.5, acceleration=0.5)

# Wait for movement to complete
while not rtde_c.isSteady():
     time.sleep(0.01)

print("Movement complete")
print(f"Final position: {rtde_r.getActualQ()}")

Starting position: [1.4412965774536133, -1.2482660573771973, 0.6950395742999476, -0.9592366975596924, 4.640990257263184, -3.063166920338766]
Movement complete
Final position: [1.4412965774536133, -1.2482660573771973, 0.6950395742999476, -0.9592366975596924, 4.640990257263184, -3.063166920338766]


## Example: Linear TCP Movement

Move the TCP in Cartesian space (straight line)

In [ ]:
# Example: Move TCP up by 50mm (0.05m) in Z-axis
# WARNING: Ensure workspace is clear before running!

# Get current TCP pose
current_pose = rtde_r.getActualTCPPose()
print(f"Starting TCP pose: {current_pose}")

# Create new pose (move up 50mm in Z)
new_pose = list(current_pose)
new_pose[1] -= 0.05  # Z-axis

# Move to new pose with linear movement
# speed: TCP speed in m/s
# acceleration: TCP acceleration in m/s^2
rtde_c.moveL(new_pose, speed=0.4, acceleration=0.3)

# Wait for movement to complete
while not rtde_c.isSteady():
     time.sleep(0.01)

print("Linear movement complete")
print(f"Final TCP pose: {rtde_r.getActualTCPPose()}")

## Advanced: Servo Mode (Real-Time Control)

For smooth, real-time control (useful for sensor-guided motion)

In [6]:
# Example: Smooth sinusoidal movement using servoJ
# WARNING: Ensure workspace is clear before running!

# # Get starting position
# start_joints = rtde_r.getActualQ()
# 
# # Servo parameters
# duration = 5.0  # seconds
# dt = 0.002  # 500 Hz control loop
# lookahead_time = 0.1
# gain = 300
# 
# # Start servo mode
# for t in range(int(duration / dt)):
#     # Create sinusoidal movement on joint 5
#     target = list(start_joints)
#     target[5] = start_joints[5] + 0.1 * math.sin(2 * math.pi * t * dt)
#     
#     # Send servo command (must be called every ~2ms)
#     rtde_c.servoJ(target, velocity=0, acceleration=0, 
#                   dt=dt, lookahead_time=lookahead_time, gain=gain)
#     
#     time.sleep(dt)
# 
# # Stop servo mode
# rtde_c.servoStop()
# print("Servo movement complete")

## Close Connection

Always stop the control interface and disconnect when done

In [7]:
# Stop any ongoing control
rtde_c.stopScript()

# Disconnect
rtde_c.disconnect()
rtde_r.disconnect()

print("Connection closed")

Connection closed


## Summary: Key RTDE Advantages for UR5e

1. **Speed:** 500 Hz update rate (25x faster than urx)
2. **Efficiency:** Sends only data packets, not full URScript programs
3. **Real-time:** Maintains robot controller's real-time properties
4. **Rich API:** Access to forces, speeds, safety status, and more
5. **Servo mode:** Smooth real-time control for advanced applications
6. **Official:** Supported by Universal Robots

For production applications or anything requiring precise, fast control on your UR5e, RTDE is the clear choice.